In [ ]:
import warnings
warnings.filterwarnings('ignore')

import random, re, time, xgboost
import numpy as np
from numpy.linalg import svd
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
#맷플랏립 그래프 주피터에서 바로 출력하기
from matplotlib import rc
rc('font', family='Malgun Gothic')      #한글 폰트설정
plt.rcParams['axes.unicode_minus']=False      #마이너스 부호 출력 설정

from scipy import stats, sparse
from scipy.stats import t, norm, chi2, chi2_contingency, skew
from sklearn.datasets import load_iris, make_classification, load_breast_cancer, load_boston, make_blobs
from sklearn import datasets, tree

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Binarizer, PolynomialFeatures

from sklearn.metrics import accuracy_score, precision_score, \
    recall_score, confusion_matrix, f1_score, classification_report,\
    precision_recall_curve, roc_auc_score, mean_squared_error, r2_score,\
    mean_absolute_error, silhouette_samples, silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, \
    GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from xgboost import plot_importance, XGBClassifier, XGBRegressor

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor

from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans

from nltk import sent_tokenize, word_tokenize
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import nltk

In [ ]:
from nltk import sent_tokenize, word_tokenize
import nltk
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from scipy import sparse

In [ ]:
# !pip install nltk
nltk.download('punkt')

### Text Tokenization

In [ ]:
text_sample = '''The Matrix is everywhere its all around us, here even in this room. 
               You can see it out your window or on your television. 
               You feel it when you go to work, or go to church or pay your taxes.'''
sentences = sent_tokenize(text=text_sample)

print(type(sentences), len(sentences))
sentences

In [ ]:
sentence = "The Matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)

print(type(words), len(words))
print(words)

In [ ]:
def tokenize_text(text):
    sent = sent_tokenize(text)
    word_tokn = [word_tokenize(i) for i in sent]
    return word_tokn

word_tokens = tokenize_text(text_sample)

print(type(word_tokens), len(word_tokens))
print(word_tokens)

### stopwords(불용어) 제거
: 일반적이고 의미없는 단어들의 집합 (문맥 이해에 도움이 안 되거나 문서간의 차이 구분에 기여하지 않는 단어들/ the, is, in, and 등)

In [ ]:
nltk.download('stopwords')

In [ ]:
print(f'영어 stop words 갯수: {len(nltk.corpus.stopwords.words("english"))}')
print(nltk.corpus.stopwords.words('english')[:20])

In [ ]:
stopwords_list = nltk.corpus.stopwords.words('english')  #불용어 목록
all_tokens = []

for s in word_tokens:   #문장별로 단어들 모두 분리
    filtered_words = []
    for word in s:
        word = word.lower()   #각 단어 소문자로 변환
        if word not in stopwords_list:
            filtered_words.append(word)   #불용어가 아니면 리스트에 추가
    all_tokens.append(filtered_words)  #불용어가 아닌 단어들로 이루어진 문장 리스트에 추가
    
print(all_tokens)

### Stemming과 Lemmatization

In [ ]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()   #활용 단어들의 어원을 찾아줌

print(stemmer.stem('working'), stemmer.stem('works'), stemmer.stem('worked'))
print(stemmer.stem('amusing'), stemmer.stem('amuses'), stemmer.stem('amused'))
print(stemmer.stem('happier'), stemmer.stem('happiest'))
print(stemmer.stem('fancier'), stemmer.stem('fanciest'))

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'), lemma.lemmatize('amuses','v'), lemma.lemmatize('amused','v'))
print(lemma.lemmatize('happier','a'), lemma.lemmatize('happiest','a'))
print(lemma.lemmatize('fancier','a'), lemma.lemmatize('fanciest','a'))
# 단어, 품사를 넣어줌 -> v(동사), a(형용사)

### 희소행렬
: 대부분의 요소가 0인 행렬<br>
큰 행렬을 효율적으로 저장하고 처리하기 위해 사용

### 희소 행렬 - COO 형식

In [ ]:
dense = np.array([[3,0,1],[0,2,0]])
dense

In [ ]:
from scipy import sparse

# 0이 아닌 데이터 추출 (3,1,2)
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 array로 생성 (0,0),(0,2),(1,0)
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용해 coo형식 희소행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))
# 데이터, 위치정보 -> 2중 행렬

In [ ]:
print(sparse_coo)
sparse_coo.toarray()

In [ ]:
dense2 = np.array([[0,0,1,0,0,5],
             [1,4,0,3,2,5],
             [0,6,0,3,0,0],
             [2,0,0,0,0,0],
             [0,0,0,7,0,8],
             [1,0,0,0,0,0]])
dense2

In [ ]:
# 0이 아닌 데이터 추출, 위치값
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])
row = np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col = np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

# coo형식으로 희소행렬 생성
sparse1 = sparse.coo_matrix((data2, (row, col)))
print(sparse1)
sparse1.toarray()

### 희소 행렬 - CSR 형식
: coo보다 조금 더 효율적으로 저장

In [ ]:
# 0이 아닌 데이터 추출, 위치값
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])
row = np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col = np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

# row배열에서 값이 바뀌는 위치
# 마지막 13은 row행의 길이(개수)
row_col_ind = np.array([0,2,7,9,10,12,13])

# CSR 형식으로 변환
saprse2 = sparse.csr_matrix((data2, col, row_col_ind))
print(saprse2)
saprse2.toarray()

## 8.2 텍스트 사전 준비 작업(텍스트 전처리) - 텍스트 정규화

### Text Tokenization

In [ ]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

text_sample = 'The Matrix is everywhere its all around us, here even in this room. \
               You can see it out your window or on your television. \
               You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text=text_sample)
print(type(sentences),len(sentences))
print(sentences)

In [ ]:
from nltk import word_tokenize

sentence = "The Matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

In [ ]:
from nltk import word_tokenize, sent_tokenize

#여러개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화 만드는 함수 생성
def tokenize_text(text):
    
    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)
    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

#여러 문장들에 대해 문장별 단어 토큰화 수행. 
word_tokens = tokenize_text(text_sample)
print(type(word_tokens),len(word_tokens))
print(word_tokens)

### Stopwords 제거

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
print('영어 stop words 갯수:',len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

In [ ]:
import nltk

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
# 위 예제의 3개의 문장별로 얻은 word_tokens list 에 대해 stop word 제거 Loop
for sentence in word_tokens:
    filtered_words=[]
    # 개별 문장별로 tokenize된 sentence list에 대해 stop word 제거 Loop
    for word in sentence:
        #소문자로 모두 변환합니다. 
        word = word.lower()
        # tokenize 된 개별 word가 stop words 들의 단어에 포함되지 않으면 word_tokens에 추가
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
    
print(all_tokens)

### Stemming과 Lemmatization

In [ ]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'),stemmer.stem('works'),stemmer.stem('worked'))
print(stemmer.stem('amusing'),stemmer.stem('amuses'),stemmer.stem('amused'))
print(stemmer.stem('happier'),stemmer.stem('happiest'))
print(stemmer.stem('fancier'),stemmer.stem('fanciest'))

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'),lemma.lemmatize('amuses','v'),lemma.lemmatize('amused','v'))
print(lemma.lemmatize('happier','a'),lemma.lemmatize('happiest','a'))
print(lemma.lemmatize('fancier','a'),lemma.lemmatize('fanciest','a'))

## 8.3 Bag of Words – BOW

### 희소 행렬 - COO 형식

In [ ]:
import numpy as np

dense = np.array( [ [ 3, 0, 1 ], [0, 2, 0 ] ] )

In [ ]:
from scipy import sparse

# 0 이 아닌 데이터 추출
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 array로 생성 
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용하여 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos,col_pos)))

In [ ]:
sparse_coo.toarray()

### 희소 행렬 – CSR 형식

In [ ]:
from scipy import sparse

dense2 = np.array([[0,0,1,0,0,5],
             [1,4,0,3,2,5],
             [0,6,0,3,0,0],
             [2,0,0,0,0,0],
             [0,0,0,7,0,8],
             [1,0,0,0,0,0]])

# 0 이 아닌 데이터 추출
data2 = np.array([1, 5, 1, 4, 3, 2, 5, 6, 3, 2, 7, 8, 1])

# 행 위치와 열 위치를 각각 array로 생성 
row_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5])
col_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0])

# COO 형식으로 변환 
sparse_coo = sparse.coo_matrix((data2, (row_pos,col_pos)))

# 행 위치 배열의 고유한 값들의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0, 2, 7, 9, 10, 12, 13])

# CSR 형식으로 변환 
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print('COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())


In [ ]:
dense3 = np.array([[0,0,1,0,0,5],
             [1,4,0,3,2,5],
             [0,6,0,3,0,0],
             [2,0,0,0,0,0],
             [0,0,0,7,0,8],
             [1,0,0,0,0,0]])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)